### NSMC language model

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import torchtext
from konlpy.tag import Mecab
from torchtext.data import Field, BucketIterator, TabularDataset, Dataset
import os

from cnn_model import CNNClassifier
from rnn_model import RNN

from torch.autograd import Variable
from attention import Attention

In [37]:
np.set_printoptions(precision=4, suppress=True)

전처리를 위해 torchtext 라이브러리를 사용한다. 텍스트를 전처리해서 단어셋 구축과 숫자화(numericalize) 역할까지 수행한다.

In [3]:
DATA_PATH = '/Users/kwang/Project/ToBigs/2018-2/Lecture_NLP/complete' 
tagger = Mecab()

USE_CUDA = torch.cuda.is_available()
DEVICE = 0 if USE_CUDA else -1 #DEVICE = 'cuda' if USE_CUDA else 'cpu'

def pad_under_five(toknized):
    """
    모델에서 5-gram 단위 필터를 사용하기 때문에
    5-gram이 안되는 문장에 <pad>로 채워준다
    """
    if len(toknized) < 5:
        toknized.extend(["<pad>"]*(5-len(toknized)))
    return toknized

한글 형태소 분석기로 mecab(konlpy)를 이용한다. windows 는 kkma 등 사용하시면 됩니다.

최소 문장 길이만큼 패딩해주는 전처리 함수를 정의한다.

### Field

In [4]:
TEXT = Field(tokenize=tagger.morphs, lower=True, init_token="<s>",
             eos_token="</s>", include_lengths=True,
             batch_first=True, preprocessing=pad_under_five) 

LABEL = Field(sequential=False,use_vocab=True,unk_token=None) 

텍스트와 정답을 Field로 정의한다. 텍스트는 mecab 형태소 단위로 분리되며 패딩 전처리 함수 또한 여기서 정의한다.  

라벨은 use_vocab 파라미터를 사용해서 일반 텍스트도 정답으로 사용될 수 있다.  

Field 내에는 vocab이라는 객체가 정의되어 있다. 다만, Field에 실제 데이터를 할당한 이후부터 사용 가능하다.

### Dataset

In [5]:
# 토큰 레벨 문장의 길이가 1 이상인 경우만 허용
train_data, test_data = TabularDataset.splits(path=DATA_PATH+'/nsmc/',
                                              train='ratings_train.txt',
                                              test='ratings_test.txt',
                                              format='tsv', 
                                              skip_header=True, 
                                              fields=[('id',None),('text',TEXT),('label',LABEL)], 
                                              filter_pred = lambda x: True if len(x.text) > 1 else False) 


TabularDataset은 실제 텍스트 파일(json, txt, csv 등)을 읽어 field에 데이터를 할당하는 역할을 한다.

이제 TEXT와 LABEL에 실제 데이터가 할당 되었다.

In [6]:
print(len(train_data), len(test_data))

150000 50000


In [7]:
TEXT.build_vocab(train_data, min_freq=2)
LABEL.build_vocab(train_data)

build_vocab이라는 메소드를 이용해서 vocab을 구축할 수 있다. 최소 빈도를 지정하여 코퍼스 구축이 가능하다.  

이제 단어셋을 아래처럼 찍어볼 수 있다.

In [8]:
len(TEXT.vocab)

29976

In [9]:
print (TEXT.vocab.itos[:100], len(TEXT.vocab))

['<unk>', '<pad>', '<s>', '</s>', '.', '이', '는', '영화', '다', '고', '하', '도', '의', '가', '은', '에', '을', '보', '한', '..', '게', ',', '들', '!', '지', '를', '있', '없', '?', '좋', '나', '었', '만', '는데', '너무', '봤', '적', '안', '정말', '로', '음', '으로', '것', '아', '네요', '재밌', '점', '어', '같', '지만', '진짜', '했', '에서', '기', '네', '않', '거', '았', '수', '되', '면', '과', '말', '연기', '인', '주', '잘', '최고', '~', '내', '평점', '이런', '던', '어요', '와', '생각', 'ㅎ', '할', '왜', '1', '겠', '스토리', '습니다', '해', '...', '드라마', '아니', '싶', '그', '사람', '듯', '함', '더', '감동', '때', '배우', '본', '까지', '좀', '뭐'] 29976


dataset 객체에서 example 멤버를 이용하면 전처리된 실제 데이터에 순차적으로 접근할 수 있다.

In [10]:
print (train_data.examples[0].text, train_data.examples[0].label)
print (train_data.examples[10].text, train_data.examples[10].label)
print (train_data.examples[100].text, train_data.examples[100].label)

['아', '더빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'] 0
['걍인피니트가짱이다', '.', '진짜', '짱', '이', '다', '♥'] 1
['신카이', '마코토', '의', '작화', '와', ',', '미유', '와', '하나', '카', '나', '가', '연기', '를', '잘', '해', '줘서', '더', '대박', '이', '였', '다', '.'] 1


In [11]:
print (TEXT.vocab.freqs['<unk>'], TEXT.vocab.freqs['.'])
print (TEXT.vocab.stoi['<unk>'], TEXT.vocab.stoi['.'])
print (TEXT.vocab.itos[0], TEXT.vocab.itos[2])
print (len(TEXT.vocab.freqs), len(TEXT.vocab.itos), len(TEXT.vocab.stoi))

0 160207
0 4
<unk> <s>
54802 29976 29976


vocab 멤버로는 freq, stoi, itos가 있으며 각각 빈도, string to int, int to string을 나타낸다.  

build_vocab에서 freq 제한을 두었기 때문에 실제 구축된 vocab의 길이가 freq 리스트의 길이보다 짧은 것을 볼 수 있다.

### Iterator

dataset으로 읽은 데이터를 이용해서 연산 가능한 형태로 만든다. iterator화 해서 학습에 유용한 loader로 변환한다.  

loader는 batch 단위로 접근이 가능하다. 문장은 numericalized된 long type 데이터이다.

In [12]:
train_loader, test_loader = BucketIterator.splits((train_data, test_data),  
                                                  sort_key=lambda x:len(x.text),
                                                  sort_within_batch=True,
                                                  repeat=False,shuffle=True,
                                                  batch_size=32,device=DEVICE)

In [13]:
for bt in train_loader:
    print(bt.text)
    break

(tensor([[     2,     38,     93,    202,     57,     82,    998,     17,
              9,    746,    595,     51,     82,     71,     29,     14,
              7,    471,     41,     11,    338,   5080,    732,    212,
           1217,     80,     82,    765,      3],
        [     2,    356,     98,  11524,     13,     78,    170,     98,
          11524,     39,   3570,    198,      4,      4,    232,      5,
             98,   1980,     49,   3209,   3245,     16,     94,     13,
             27,      8,      4,    230,      3],
        [     2,     40,      4,     19,    502,   1151,     28,     28,
              7,     25,    895,     39,    287,     14,    172,     28,
          11741,   1769,     22,     14,   4793,    103,     32,     14,
             77,     90,      4,      4,      3],
        [     2,    375,     10,     20,    188,    236,     15,    327,
          15816,     74,    959,     20,     35,     72,      7,    146,
              4,   2298,     61,    257,     6

In [21]:
class RNN(nn.Module):
    def __init__(self,input_size,embed_size,hidden_size,output_size,
                 num_layers=1,bidirec=False,vocab=None):
        super(RNN,self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if bidirec:
            self.num_directions = 2
        else:
            self.num_directions = 1
            
        self.embed = nn.Embedding(input_size,embed_size)

        self.lstm = nn.LSTM(embed_size,hidden_size,num_layers,batch_first=True,bidirectional=bidirec)
        self.linear = nn.Linear(hidden_size*self.num_directions,output_size)
        #self.attention = Attention(self.hidden_size*self.num_directions, method='general')
        
    def init_hidden(self,batch_size):
        # (num_layers * num_directions, batch_size, hidden_size)
        
        hidden = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size)).cuda()
        cell = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size)).cuda()
        return hidden, cell

    def forward(self,inputs, encoder_length=None):
        """
        inputs : B,T
        """
        embed = self.embed(inputs) # word vector indexing
        hidden, cell = self.init_hidden(inputs.size(0)) # initial hidden,cell
        
        output, h = self.lstm(embed,(hidden,cell)) # output : Batch,Time,Hidden (32, 9, 200)
        hidden, cell = h # hidden : L, B, H
        
        hidden = hidden[-self.num_directions:] # (num_directions,B,H)
        hidden = torch.cat([h for h in hidden],1) #.unsqueeze(0) # (1,B,2H) (1, 32, 200)
        
        #print (output.size(), hidden.size(), len(encoder_length))
        #print (encoder_length, type(encoder_length[0]))
        
        # Many-to-One
        output = self.linear(hidden) # last hidden
        output = output.squeeze(1)
        
        return F.log_softmax(output, dim=1)

In [22]:
EPOCH = 10
BATCH_SIZE = 32
EMBED = 200
KERNEL_DIM = 100
LR = 0.001
output_size = 2

# model = CNNClassifier(len(TEXT.vocab), EMBED, 1, KERNEL_DIM, KERNEL_SIZES)
model = RNN(len(TEXT.vocab), EMBED, KERNEL_DIM, output_size, num_layers=2,
            bidirec=True, vocab=TEXT.vocab)

loss_function = nn.NLLLoss() # nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5], gamma=0.1)

if USE_CUDA:
    model = model.cuda()

In [ ]:
### train
model.train()
for epoch in range(EPOCH):
    losses=[]
    scheduler.step()
    for i,batch in enumerate(train_loader):
        inputs, lengths = batch.text
        targets = batch.label
        # print (inputs.size())

        if USE_CUDA:
            inputs = inputs.cuda()
            targets = targets.cuda()
        
        model.zero_grad()
        
        preds = model(inputs, encoder_length=lengths.tolist())
        # print (preds, targets)
        # print (preds.squeeze(1).size(), targets.size())

        loss = loss_function(preds,targets)
        losses.append(loss.item())

        # exit()
        loss.backward()
        optimizer.step()

        if i % 1000 == 0:
            print("epoch : %d mean_loss : %.3f , lr : %.5f" % (epoch,np.mean(losses), scheduler.get_lr()[0]))
            losses=[]

torch.save(model.state_dict(), './model/nsmc_lm.pth')

In [ ]:
### evaluate
model.eval()
num_hit=0

for i,batch in enumerate(test_loader):
    inputs, lengths = batch.text
    targets = batch.label
    
#    print ('\n',inputs.size())
#    print (inputs)
#    break

    if USE_CUDA:
        inputs = inputs.cuda()
        targets = targets.cuda()

    output = model(inputs, lengths)

    preds = output.max(1, keepdim=True)[1]
#     print (preds.size())
#     print (preds)
#     print (targets)
#     preds = preds*5
#     preds = preds.round()
    num_hit+=torch.eq(preds.squeeze(),targets.squeeze()).sum().item() 

print('test accuracy: %.4f%%'%(num_hit/len(test_data)*100))

In [ ]:
### test
test_inputs = ["개별적인 장면이 좋았다.", "존맛탱!", "헐 진짜 개별로다..", 
               "진짜 너무 재밌는 영화다 오랜만에","오..이건 진짜 봐야함", 
               "진짜 쓰레기 같은 영화","노잼","존잼","꾸울잼","핵노잼",'또 보고싶다', 
               '꼬옥 봐야한다.. 진짜..', '나만 보기 아깝다', '돈이 아깝다', '나만 보기 억울하다', 
               '나만 당할 수 없다', '너도 봐야한다', '혼자 본게 정말 후회된다. 이건 꼭 같이 봐야한다.', 
               '재미없어요...', '꾸르르르르르르잼', '꾸르르르잼', '꾸르잼', '이 영화를 보고 암이 나았습니다.']  

for test_input in test_inputs:
    tokenized = tagger.morphs(test_input)
    length = len(tokenized)
    tokenized = pad_under_five(tokenized)
    input_, lengths = TEXT.numericalize(([tokenized], length), device=DEVICE)
    if USE_CUDA: input_ = input_.cuda()
    
#     output, attn_weights = model(input_, [lengths.tolist()])
    output = model(input_, [lengths.tolist()])
#     print (output)
    prediction = output.max(1, keepdim=True)[1]
#     print (prediction)

    if prediction[0][0] == 1:
        print(test_input,"\033[1;01;36m" + '긍정' + "\033[0m")
    else:
        print(test_input,"\033[1;01;31m" + '부정' + "\033[0m")
    print ()
    
print (LABEL.vocab.itos)


In [ ]:
torch.save(TEXT.vocab, './model/news_field.pth')

In [ ]:
print (LABEL.numericalize(['스포츠']))

In [ ]:
vocaburay = torch.load('./model/vocab.pth')

In [ ]:
print (vocaburay.itos[:10])
TEST = Field(tokenize=tagger.morphs,lower=True,include_lengths=False,batch_first=True,preprocessing=pad_under_five)
TEST.build_vocab()
TEST.vocab = vocaburay
TEST.numericalize(test_inputs[0])